In [4]:
# imports
import os
import requests
from dotenv import load_dotenv

from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from scraper import fetch_website_contents

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

If you get a "Name Error" - have you run all cells from the top down? Head over to the Python Foundations guide for a bulletproof way to find and fix all Name Errors.

If that doesn't fix it, head over to the [troubleshooting](../../setup/troubleshooting.ipynb) notebook for step by step code to identify the root cause and fix it!

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [21]:
# A class to represent a Webpage

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [5]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [22]:
# Let's try one out

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## OK onwards with our first project

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)

# Sharing your code

I'd love it if you share your code afterwards so I can share it with others! You'll notice that some students have already made changes (including a Selenium implementation) which you will find in the community-contributions folder. If you'd like add your changes to that folder, submit a Pull Request with your new versions in that folder and I'll merge your changes.

If you're not an expert with git (and I am not!) then GPT has given some nice instructions on how to submit a Pull Request. It's a bit of an involved process, but once you've done it once it's pretty clear. As a pro-tip: it's best if you clear the outputs of your Jupyter notebooks (Edit >> Clean outputs of all cells, and then Save) for clean notebooks.

Here are good instructions courtesy of an AI friend:  
https://chatgpt.com/share/677a9cb5-c64c-8012-99e0-e06e88afd293

In [31]:
#Parse webpages which is designed using JavaScript heavely
# download the chorme driver from here as per your version of chrome - https://developer.chrome.com/docs/chromedriver/downloads
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options

PATH_TO_CHROME_DRIVER = 'C:/Program Files/Google/chromedriver-win32/chromedriver.exe'

class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url

        options = Options()

        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")

        service = Service(PATH_TO_CHROME_DRIVER)
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(url)

        page_source = driver.page_source
        driver.quit()

        soup = BeautifulSoup(page_source, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.get_text(separator="\n", strip=True)

In [35]:

system_prompt = "You are a journalist from a basketball sports section. You are interested in statistics, and in giving a clear view of what happened in the game."


In [42]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "The contents of this website is as follows; \
You should give a summary of what happened in the game, and what the different teams did. You should highlight particular players, both on the good and the bad side, and try your best to tell a story of the game. Additionally, give me an idea of how Neemias Queta did and how he impacted the game. \n\n"
    user_prompt += website.text
    return user_prompt

In [43]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]
def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [44]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [45]:
display_summary("https://www.nba.com/game/bos-vs-mil-0022501206/box-score#box-score")


On December 11, 2025, the Boston Celtics faced off against the Milwaukee Bucks in a high-stakes matchup that saw the Bucks triumph with a score of 116 to 101. This victory showcased the prowess of several players, particularly Kyle Kuzma, who delivered an outstanding performance.

### Game Summary
From the outset, the Bucks established a commanding presence. Milwaukee's offense was spearheaded by Kyle Kuzma, who delivered a breathtaking 31 points on an impressive 76.5% shooting. Additionally, Bobby Portis contributed significantly, adding 27 points with a stellar shooting performance, which kept the Celtics struggling to keep pace.

Boston's Jaylen Brown did his best to rally his team, pouring in 30 points with decent efficiency, but the overall contribution from the Celtics' squad fell short. The Bucks held the momentum throughout the game, steadily building their lead and maintaining control over the pace of play.

### Key Players
- **Kyle Kuzma (MIL)**: The standout of the night, Kuzma not only scored prolifically but also contributed vital rebounds, collecting 6 boards. His efficient shooting and ability to find space allowed him to dominate on the offensive end.
  
- **Bobby Portis (MIL)**: Portis’s performance provided the necessary support for Kuzma, allowing the Bucks to stretch the lead and secure the victory. He was on fire from the field, showcasing his scoring ability with an 84.6% field goal rate.

- **Jaylen Brown (BOS)**: Despite the loss, Brown's tenacity on offense kept the Celtics in the game for longer than they perhaps should have been. He shot 58.8% from the field, including hitting 8 of 9 free throws. However, he had minimal support from his teammates.

### Celtics' Struggles
The Celtics struggled against the Bucks' defense and failed to find consistent scoring options. Players like Payton Pritchard and Derrick White had poor shooting nights, combining for only 22 points on dismal percentages. Their inability to make timely baskets compounded the energy the Bucks brought to Fiserv Forum.

### Neemias Queta's Impact
Neemias Queta had a quiet but impactful performance in his 25 minutes on the floor, finishing with 9 points and 10 rebounds. His presence in the paint provided Boston with a much-needed rebounding edge, as they totaled 39 rebounds, compared to Milwaukee's 40. However, despite his contributions, Queta struggled to create scoring opportunities for himself and was somewhat overshadowed by the scoring woes that plagued the Celtics.

In summary, the Bucks’ collective effort led to a convincing win against the Celtics, with Kuzma’s stellar performance front and center. The Celtics will need to regroup and find solutions, especially in terms of shooting efficiency, to bounce back after this defeat.